In [1]:
# needed for functions
import numpy as np
import time
from copy import deepcopy

# qiskit imports
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise

# tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

In [2]:
# Load IBMQ account
provider = qiskit.IBMQ.load_account()
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx4') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [3]:
# expected density matrix
a = QuantumRegister(1, 'a')
b = QuantumRegister(1, 'b')
bell = QuantumCircuit(a, b)

bell.h(a[0])
bell.cx(a[0], b[0])
print(bell)

job = qiskit.execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_statevector(bell)
print(psi_bell)

        ┌───┐     
a_0: |0>┤ H ├──■──
        └───┘┌─┴─┐
b_0: |0>─────┤ X ├
             └───┘
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [4]:
# create the actual circuit
aqs = QuantumRegister(3, 'a')
bqs = QuantumRegister(3, 'b')
bell = QuantumCircuit(aqs, bqs)
bell.h(aqs[2])
bell.cx(aqs[2], bqs[2])
print(bell)

                  
a_0: |0>──────────
                  
a_1: |0>──────────
        ┌───┐     
a_2: |0>┤ H ├──■──
        └───┘  │  
b_0: |0>───────┼──
               │  
b_1: |0>───────┼──
             ┌─┴─┐
b_2: |0>─────┤ X ├
             └───┘


In [6]:
# generate circuit and run on simulator
t = time.time()
# generate the state tomography circuits. only pass in the 
# registers we want to measure (in this case aqs[2] and bqs[2])
qst_bell = state_tomography_circuits(bell, [aqs[2], bqs[2]])
job = qiskit.execute(qst_bell, provider.get_backend('ibmq_qasm_simulator'), shots=5000)
print('Exec   - Time taken:', time.time() - t)

t = time.time()
tomo_bell = StateTomographyFitter(job.result(), qst_bell)
print('Fitter - Time taken:', time.time() - t)

Exec   - Time taken: 0.20228981971740723
Fitter - Time taken: 9.213082075119019


In [8]:
# perform the tomography fit
# which outputs a density matrix
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity = ', F_bell)

Fit Fidelity =  0.9959430949857523


### Fitting larger circuits

Trying to fit the entire circuit and not just the 2 qubits in question and see the time usage

In [9]:
# expected density matrix
a = QuantumRegister(3, 'a')
b = QuantumRegister(3, 'b')
bell = QuantumCircuit(a, b)

bell.h(a[2])
bell.cx(a[2], b[2])
print(bell)

job = qiskit.execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_statevector(bell)
print(psi_bell)

                  
a_0: |0>──────────
                  
a_1: |0>──────────
        ┌───┐     
a_2: |0>┤ H ├──■──
        └───┘  │  
b_0: |0>───────┼──
               │  
b_1: |0>───────┼──
             ┌─┴─┐
b_2: |0>─────┤ X ├
             └───┘
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.   

In [15]:
# generate circuit and run on simulator
t = time.time()
# generate the state tomography circuits. only pass in the 
# registers we want to measure (in this case aqs[2] and bqs[2])
qst_bell = state_tomography_circuits(bell, [aqs[0], aqs[1], aqs[2], bqs[0], bqs[1], bqs[2]])
# job = qiskit.execute(qst_bell, provider.get_backend('ibmq_qasm_simulator'), shots=5000)
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
result = job.result()
print('Execution     - Time taken:', time.time() - t)



t = time.time()
tomo_bell = StateTomographyFitter(result, qst_bell)
print('Create Fitter - Time taken:', time.time() - t)

Exec   - Time taken: 30.229041814804077
Fitter - Time taken: 0.5484452247619629


In [16]:
# perform the tomography fit
# which outputs a density matrix
t = time.time()
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit - Time taken:', time.time() - t)
print('Fit Fidelity = ', F_bell)

Fit Fidelity =  0.9936878804974898
